In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
import os
import time
import sys
import ast
#from tqdm.auto import tqdm, trange
from tqdm import tqdm
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
from UDCUtils import UDCUtils
utils = UDCUtils()

In [4]:
pincone_api_key = utils.get_pinecone_api_key()
pincone_api_key

'fcd3e082-248a-4a31-b1c0-80c255c9d687'

In [5]:
openai_api_key = utils.get_openai_api_key()
openai_api_key

'sk-proj-SZAw4DI5L2Zd93SJBrzFT3BlbkFJB5JAmm8RCxmWB0JRAuDT'

In [6]:
if pincone_api_key==None:
    pincone_api_key = "a85f0b7c-6ce3-458d-aa18-168f39e4420b" ##default key
pincone_api_key

'fcd3e082-248a-4a31-b1c0-80c255c9d687'

In [7]:
#!wget -q --show-progress -O all-the-news-3.zip "https://www.dropbox.com/scl/fi/wruzj2bwyg743d0jzd7ku/all-the-news-3.zip?rlkey=rgwtwpeznbdadpv3f01sznwxa&dl=1"

all-the-news-3.zip  100%[===================>] 166.04M  7.05MB/s    in 40s     


In [8]:
#!unzip all-the-news-3.zip

Archive:  all-the-news-3.zip
replace all-the-news-3.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [9]:
file_name = "all-the-news-3.csv"
df = pd.read_csv(file_name, nrows=100)
df.head()

,date,year,month,day,author,title,article,url,section,publication
0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox
1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider
2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters
3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters
4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ


In [10]:
#%pip install --upgrade --quiet tqdm

In [11]:
openai_client = OpenAI(api_key=openai_api_key)

index_name = utils.create_dlai_index_name("dev-001")
print(index_name)

dev-001-118d1-7f10-4f3c-a4a6-41d7caaf84cc:fx


In [12]:
pinecone_obj = Pinecone(api_key=pincone_api_key)

index_name = index_name[:-3]

if index_name in pinecone_obj.list_indexes():
    pinecone_obj.delete_index(index_name)

In [13]:
print(index_name)

dev-001-118d1-7f10-4f3c-a4a6-41d7caaf84cc


In [14]:
pinecone_obj.create_index(name=index_name, dimension=1536, metric="cosine",
                         spec=ServerlessSpec(cloud="aws", region="us-east-1"))

index = pinecone_obj.Index(index_name)

In [15]:
def get_embeddings(articles, model_name = "text-embedding-ada-002"):
    return (openai_client.embeddings.create(input=articles, model=model_name))

In [16]:
total_rows = 20000
chunksize=400
progress_bar = tqdm(total=total_rows)

chunks = pd.read_csv(file_name, chunksize=chunksize, nrows=total_rows)

prepped=[]
chunk_num = 0

for chunk in chunks:
    print(chunk_num)
    titles = chunk["title"].tolist()
    embeddings = get_embeddings(titles)
    prepped = [
        {"id": str(chunksize*chunk_num+i),
        "metadata": {"title":titles[i]},
        "values":embeddings.data[i].embedding} for i in range(len(titles))
    ]
    chunk_num = chunk_num + 1
    #print(len(prepped))
    if len(prepped)>=200:
        index.upsert(prepped)
        prepped=[]
    progress_bar.update(len(chunk))
#df.info()

if len(prepped)>0:
    index.upsert(prepped)

  0%|                                                 | 0/20000 [00:00<?, ?it/s]

0


  2%|▊                                      | 400/20000 [00:13<11:25, 28.58it/s]

1


  4%|█▌                                     | 800/20000 [00:23<09:11, 34.82it/s]

...

 96%|███████████████████████████████████▌ | 19200/20000 [06:34<00:16, 47.90it/s]

48


 98%|████████████████████████████████████▎| 19600/20000 [06:40<00:07, 51.40it/s]

49


100%|█████████████████████████████████████| 20000/20000 [06:47<00:00, 54.75it/s]

In [17]:
print(chunk_num)

50


In [18]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 20000}},
 'total_vector_count': 20000}

In [19]:
def get_recommendations(pinecone_index, search_term, top_k=10):
    embedding = get_embeddings([search_term]).data[0].embedding
    response = pinecone_index.query(vector=embedding, top_k = top_k, include_metadata = True)
    return response

In [20]:
response = get_recommendations(index, "president obama")
#print(response.matches.score)

for records in response.matches:
    print(f"{records.score}:{records.metadata["title"]}")

0.862794399:President Obama has a new plan to fight the opioid epidemic
0.86267668:“Our democracy is at stake”: Obama delivers his first post-presidency campaign speech
0.859710515:Obama meets with national security team on Syria, Islamic State
0.858792126:Obama’s parting message is a warning for Donald Trump
0.857344:Barack Obama just stepped off the sidelines to defend Obamacare
0.856670141:Obama's been a much more effective communicator than he gives himself credit for
0.854823351:Watch President Obama dance the tango in Argentina
0.852502:President Obama to Court Emerging Donor Class of the Democratic Party at SXSW
0.852015376:America will miss Barack Obama's decency
0.850985706:Prez Obama -- I Support Colin Kaepernick's National Anthem Protest (VIDEO)


### we will search articles that contain these term

In [21]:
index_name = utils.create_dlai_index_name("dl-ai-002")
index_name = index_name[:-3]
print(index_name)

if index_name in pinecone_obj.list_indexes():
    pinecone_obj.delete_index(index_name)

pinecone_obj.create_index(name=index_name,
                         dimension=1536,
                         metric="cosine",
                         spec=ServerlessSpec(cloud="aws", region="us-east-1"))

index_article = pinecone_obj.Index(index_name)

dl-ai-002-118d1-7f10-4f3c-a4a6-41d7caaf84cc


100%|█████████████████████████████████████| 20000/20000 [07:00<00:00, 54.75it/s]

In [22]:
#%pip install ipywidgets==7.7

In [23]:
from tqdm import tqdm

In [24]:
from time import sleep
for i in tqdm(range(0,100)):
    sleep(0.01)


100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 79.75it/s]


In [25]:
#help(index_article.delete)
index_article.delete(delete_all=True)

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 16 Aug 2024 05:56:38 GMT', 'Content-Type': 'application/json', 'Content-Length': '55', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '71', 'x-pinecone-request-id': '7243761985264142297', 'x-envoy-upstream-service-time': '72', 'server': 'envoy'})
HTTP response body: {"code":5,"message":"Namespace not found","details":[]}


In [26]:
prepped = []

chunksize=200
total_rows=10000

print(file_name)
chunks = pd.read_csv(file_name, chunksize=chunksize, nrows=total_rows)

#define a text splitter for splitting articles into tokens
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunksize,
    chunk_overlap=20
)

embed_cnt=1
chunk_num=0
for chunk in chunks:
    print(chunk_num)
    titles = chunk["title"].to_list()
    #title_embedding = get_embeddings(titles)
    articles = chunk["article"].to_list()
    #print(len(titles))
    #print(len(articles))
    for i, article in tqdm(enumerate(articles)):
        #print(f"article>>>{i}")
        if article == None:
            articles[i] = ""
        #embeddings = get_embeddings(articles)
        else:
            embed_cnt = chunksize * chunk_num * i
            tokens = text_splitter.split_text(str(article))
            embeddings = get_embeddings(tokens)
            #print(f"embed_cnt:{embed_cnt}")
            #print(f"len(embeddings):{len(embeddings.data)}")
            prepped.extend([{
            "id": str(embed_cnt+j),
            "metadata": {"title":titles[i]},
            "values":embeddings.data[j].embedding} for j in range(len(tokens))
                      ])
            #print(len(prepped))
        if(len(prepped)>=200):
            print(f"Upserting in index for chunk {chunk_num}")
            index_article.upsert(prepped)
            prepped.clear()
        
    chunk_num = chunk_num + 1

all-the-news-3.csv
0



0it [00:00, ?it/s]
1it [00:01,  1.61s/it]
2it [00:02,  1.15s/it]
3it [00:03,  1.09it/s]
4it [00:04,  1.01it/s]
5it [00:04,  1.13it/s]
6it [00:05,  1.14it/s]
7it [00:06,  1.08it/s]
8it [00:08,  1.16s/it]
9it [00:09,  1.05s/it]
10it [00:10,  1.15s/it]

Upserting in index for chunk 0



11it [00:16,  2.52s/it]
12it [00:17,  2.16s/it]
13it [00:18,  1.76s/it]
14it [00:19,  1.50s/it]
15it [00:19,  1.23s/it]
16it [00:21,  1.36s/it]
17it [00:23,  1.49s/it]
18it [00:24,  1.39s/it]
19it [00:25,  1.27s/it]
20it [00:26,  1.24s/it]
21it [00:27,  1.06s/it]

Upserting in index for chunk 0


...

Upserting in index for chunk 2



148it [03:25,  2.30s/it]
149it [03:27,  2.06s/it]
150it [03:28,  1.82s/it]
151it [03:29,  1.54s/it]

Upserting in index for chunk 2


151it [03:36,  1.43s/it]


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 16 Aug 2024 06:15:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '94', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '3129', 'x-pinecone-request-id': '8324708836285359102', 'x-envoy-upstream-service-time': '132', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Request size 3MB exceeds the maximum supported size of 2MB","details":[]}


In [27]:
index_article.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 7023}},
 'total_vector_count': 7023}

In [30]:
reco = get_recommendations(index_article, 'President Obama', top_k=10)
seen = {}
for r in reco.matches:
    title = r.metadata['title']
    if title not in seen:
        print(f'{r.score} : {title}')
        seen[title] = '.'

0.862866521 : How tech workers in the government could make everyone’s life easier: ‘Get rid of the paper’
0.845256448 : 'Cut it out,' Obama told Putin on cyber attacks
0.841712117 : The game theory behind Mitch McConnell's Supreme Court strategy
0.828852713 : Full transcript of Donald Trump’s acceptance speech at the RNC
0.826500714 : White House plans new social media summit as Congress raises questions
0.825761318 : Anti-Trump protesters gather for third night, one shot in Portland
0.825548 : In Trump We Trust: Inauguration prompts celebration in Russia
0.824859381 : Midterm elections 2018 live results: the House seats Democrats flipped
